In [1]:
import fitz
import pandas as pd
from datetime import datetime
import logging
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)

logging.basicConfig(level=logging.DEBUG)
# filename = 'data/fwil_dhi_me_results_semi.pdf'
# filename = 'data/fwil_dhi_me_results_qr.pdf'
# filename = 'data/fwil_dhi_me_results_f.pdf'
# filename = 'data/fwil_dhi_me_results_tt.pdf'
# filename = 'data/biel_dhi_me_results_tt.pdf'
filename = 'data/leog_dhi_me_results_tt.pdf'


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Time trial split pdfs are different format than the qualifiers, semis and finals pdfs.

In [2]:
import pandas as pd
import re
import fitz  # PyMuPDF

def parse_timed_training_data_final(lines):
    data = []
    current_rider = {}
    run_number = 0
    collect_splits = False
    start_collecting = False  # Flag to start collecting data

    for line in lines:
        line = line.strip()
        if 'Nr Name / UCI MTB Team' in line:  # This marks the start of rider data
            start_collecting = True
            continue
        if not start_collecting:
            continue

        if line.endswith('.') or (re.match(r'^\d+ [A-Z]', line) and not current_rider):
            # Resetting for a new rider entry
            if current_rider:
                data.append(current_rider.copy())
                current_rider = {}
                run_number = 0
                collect_splits = False
            current_rider['Rank'] = line[:-1] if line.endswith('.') else None
            continue

        if re.match(r'^\d+ [A-Z]', line):  # Captures the number and name
            parts = line.split(maxsplit=1)
            current_rider['Number'] = parts[0]
            current_rider['Name'] = parts[1]
        elif line.isupper():  # Handles team and nationality
            if any(ext in line for ext in ['TEAM', 'FACTORY', 'RACING', 'GRAVITY']):
                current_rider['Team'] = line
            else:
                current_rider['NAT'] = line
        elif re.match(r'^\d+\.\d+$', line):  # Handles speed for a run
            run_number += 1
            current_rider[f'Run{run_number}_Speed'] = line
            collect_splits = True
            current_rider[f'Run{run_number}_Splits'] = []  # Prepare to collect splits
        elif collect_splits and (re.match(r'\d+:\d+\.\d+$', line) or re.match(r'\d+\.\d+$', line)):  # Captures splits
            current_rider[f'Run{run_number}_Splits'].append(line)
        elif collect_splits and re.match(r'\+\d+\.\d+$', line):  # Handles the delta time
            current_rider[f'Run{run_number}_Delta'] = line
            collect_splits = False  # Stop collecting splits for this run

    # Append the last rider if not already appended
    if current_rider:
        data.append(current_rider)

    # Convert to DataFrame and filter entries with missing crucial data
    df = pd.DataFrame(data)
    df = df[df['Number'].notna() & df['Name'].notna()]  # Ensure every entry has at least a number and a name
    return df

# Open the PDF and read its content
# filename = '/mnt/data/biel_dhi_me_results_tt.pdf'
doc = fitz.open(filename)

all_text = []
for page in doc:
    text = page.get_text("text")
    all_text.extend(text.split('\n'))

# Parse the data directly from PDF text
df_timed_training_final = parse_timed_training_data_final(all_text)
print(df_timed_training_final.tail())  # Display the last few entries to verify the parsing


     NAT Rank Number                Name                    Team Run1_Speed  \
171  AUT  171    144  ROHRMOSER Valentin                     NaN     50.847   
172  IRL  172    179          ENNIS Ross                     NaN     49.405   
173  AUT  173    182       BUHL Kilian *                     NaN     54.000   
174  USA  174    192    KITCHEN Nathan *  THE GRAVITY COLLECTIVE     54.748   
175  AUT  175    108       TRUMMER David       MS INTENSE RACING     48.043   

                                           Run1_Splits Run2_Speed  \
171  [0:38.922, 1:31.035, 2:11.651, 6:08.427, 16:29...     50.688   
172  [0:40.411, 6:09.765, 6:51.822, 16:32.774, 17:1...     51.658   
173  [0:38.578, 7:33.996, 8:15.256, 22:49.572, 28:4...     52.614   
174                     [0:38.494, 3:25.801, 4:05.617]     53.589   
175  [1:17.760, 3:01.427, 3:46.039, 31:05.700, 32:3...     49.330   

                                           Run2_Splits Run3_Speed  \
171  [0:38.642, 7:49.020, 8:30.314, 23:27

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_timed_training_final

['0:45.070', '1:43.159', '2:12.260', '4:41.049', '5:21.632']

Create and manipulate


In [22]:
import pandas as pd
import re
import fitz  # PyMuPDF

def parse_timed_training_data_final(lines):
    data = []
    current_rider = {}
    run_number = 0
    collect_splits = False
    start_collecting = False  # Flag to start collecting data

    for line in lines:
        line = line.strip()
        if 'Nr Name / UCI MTB Team' in line:  # This marks the start of rider data
            start_collecting = True
            continue
        if not start_collecting:
            continue

        if line.endswith('.') or (re.match(r'^\d+ [A-Z]', line) and not current_rider):
            # Resetting for a new rider entry
            if current_rider:
                data.append(current_rider.copy())
                current_rider = {}
                run_number = 0
                collect_splits = False
            current_rider['Rank'] = line[:-1] if line.endswith('.') else None
            continue

        if re.match(r'^\d+ [A-Z]', line):  # Captures the number and name
            parts = line.split(maxsplit=1)
            current_rider['Number'] = parts[0]
            current_rider['Name'] = parts[1]
        elif line.isupper():  # Handles team and nationality
            if any(ext in line for ext in ['TEAM', 'FACTORY', 'RACING', 'GRAVITY']):
                current_rider['Team'] = line
            else:
                current_rider['NAT'] = line
        elif re.match(r'^\d+\.\d+$', line):  # Handles speed for a run
            run_number += 1
            current_rider[f'Run{run_number}_Speed'] = line
            collect_splits = True
            current_rider[f'Run{run_number}_Splits'] = []  # Prepare to collect splits
        elif collect_splits and (re.match(r'\d+:\d+\.\d+$', line) or re.match(r'\d+\.\d+$', line)):  # Captures splits
            current_rider[f'Run{run_number}_Splits'].append(line)
        elif collect_splits and re.match(r'\+\d+\.\d+$', line):  # Handles the delta time
            current_rider[f'Run{run_number}_Delta'] = line
            collect_splits = False  # Stop collecting splits for this run

    # Append the last rider if not already appended
    if current_rider:
        data.append(current_rider)

    # Convert to DataFrame and filter entries with missing crucial data
    df = pd.DataFrame(data)
    df = df[df['Number'].notna() & df['Name'].notna()]  # Ensure every entry has at least a number and a name
    return df

# Open the PDF and read its content
filename = 'data/biel_dhi_me_results_tt.pdf'
doc = fitz.open(filename)

all_text = []
for page in doc:
    text = page.get_text("text")
    all_text.extend(text.split('\n'))

# Parse the data directly from PDF text
df_timed_training_final = parse_timed_training_data_final(all_text)

# Extract runs and splits into a new structure
run_data = []
for _, row in df_timed_training_final.iterrows():
    for run in range(1, 4):
        run_splits = row.get(f'Run{run}_Splits', [])
        if run_splits:
            run_entry = {
                'Number': row['Number'],
                'Name': row['Name'],
                'Run': run,
                'Speed': row.get(f'Run{run}_Speed', None),
                'Splits': run_splits if isinstance(run_splits, list) else []
            }
            run_data.append(run_entry)

# Convert to DataFrame
df_runs = pd.DataFrame(run_data)

# Define function to calculate sector times
def calculate_sector_times(splits):
    sectors = []
    previous_split_time = 0
    for split in splits:
        # Convert split time to seconds
        if ':' in split:
            minutes, seconds = map(float, split.split(':'))
            split_time = minutes * 60 + seconds
        else:
            split_time = float(split)
        sector_time = split_time - previous_split_time
        sectors.append(sector_time)
        previous_split_time = split_time
    return sectors

# Add sector times to the DataFrame
df_runs['Sector_Times'] = df_runs['Splits'].apply(lambda x: calculate_sector_times(x) if isinstance(x, list) else [])

# Ensure each run has 5 sectors
max_splits = 5

def pad_sector_times(sector_times):
    if len(sector_times) < max_splits:
        sector_times.extend([None] * (max_splits - len(sector_times)))
    return sector_times

df_runs['Sector_Times'] = df_runs['Sector_Times'].apply(pad_sector_times)

# Flatten the sector times into separate columns
for i in range(max_splits):
    df_runs[f'Sector_{i+1}_Time'] = df_runs['Sector_Times'].apply(lambda x: x[i])

# Flatten the splits into separate columns
for i in range(max_splits):
    df_runs[f'Split_{i+1}_Time'] = df_runs['Splits'].apply(lambda x: x[i] if i < len(x) else None)

# Drop the original splits and sector times columns
df_runs.drop(columns=['Splits', 'Sector_Times'], inplace=True)

# Display the final DataFrame
print(df_runs.head())

# Rank each split independently
for i in range(max_splits):
    df_runs[f'Split_{i+1}_Rank'] = df_runs[f'Split_{i+1}_Time'].rank(method='min')

# Rank each sector independently
for i in range(max_splits):
    df_runs[f'Sector_{i+1}_Rank'] = df_runs[f'Sector_{i+1}_Time'].rank(method='min')

# Display the DataFrame with ranks
print(df_runs.head())

# Calculate the total time for each run by summing the sector times
df_runs['Total_Time'] = df_runs[[f'Sector_{i+1}_Time' for i in range(max_splits)]].sum(axis=1)

# Rank the overall performance of each run based on the total time
df_runs['Overall_Rank'] = df_runs['Total_Time'].rank(method='min')

# Display the final DataFrame with ranks and total times
print(df_runs.head())
print(df_runs.tail())


  Number          Name  Run   Speed  Sector_1_Time  Sector_2_Time  \
0      8  BROSNAN Troy    1  53.824         45.070         58.089   
1      8  BROSNAN Troy    2  57.353         44.328         54.851   
2      8  BROSNAN Troy    3  58.137         43.300         52.757   
3     27  THIRION Rémi    1  55.847         45.759        132.110   
4     27  THIRION Rémi    2  53.578         42.594         53.286   

   Sector_3_Time  Sector_4_Time  Sector_5_Time Split_1_Time Split_2_Time  \
0         29.101        148.789         40.583     0:45.070     1:43.159   
1        183.361         48.301         27.531     0:44.328     1:39.179   
2         27.206         47.028         26.956     0:43.300     1:36.057   
3         28.022         56.811        337.480     0:45.759     2:57.869   
4         26.938         51.671         26.981     0:42.594     1:35.880   

  Split_3_Time Split_4_Time Split_5_Time  
0     2:12.260     4:41.049     5:21.632  
1     4:42.540     5:30.841     5:58.372  

In [36]:
import pandas as pd
import re
import fitz  # PyMuPDF

def parse_timed_training_data_final(lines):
    data = []
    current_rider = {}
    run_number = 0
    collect_splits = False
    start_collecting = False  # Flag to start collecting data

    for line in lines:
        line = line.strip()
        if 'Nr Name / UCI MTB Team' in line:  # This marks the start of rider data
            start_collecting = True
            continue
        if not start_collecting:
            continue

        if line.endswith('.') or (re.match(r'^\d+ [A-Z]', line) and not current_rider):
            # Resetting for a new rider entry
            if current_rider:
                data.append(current_rider.copy())
                current_rider = {}
                run_number = 0
                collect_splits = False
            current_rider['Rank'] = line[:-1] if line.endswith('.') else None
            continue

        if re.match(r'^\d+ [A-Z]', line):  # Captures the number and name
            parts = line.split(maxsplit=1)
            current_rider['Number'] = parts[0]
            current_rider['Name'] = parts[1]
        elif line.isupper():  # Handles team and nationality
            if any(ext in line for ext in ['TEAM', 'FACTORY', 'RACING', 'GRAVITY']):
                current_rider['Team'] = line
            else:
                current_rider['NAT'] = line
        elif re.match(r'^\d+\.\d+$', line):  # Handles speed for a run
            run_number += 1
            current_rider[f'Run{run_number}_Speed'] = line
            collect_splits = True
            current_rider[f'Run{run_number}_Splits'] = []  # Prepare to collect splits
        elif collect_splits and (re.match(r'\d+:\d+\.\d+$', line) or re.match(r'\d+\.\d+$', line)):  # Captures splits
            current_rider[f'Run{run_number}_Splits'].append(line)
        elif collect_splits and re.match(r'\+\d+\.\d+$', line):  # Handles the delta time
            current_rider[f'Run{run_number}_Delta'] = line
            collect_splits = False  # Stop collecting splits for this run

    # Append the last rider if not already appended
    if current_rider:
        data.append(current_rider)

    # Convert to DataFrame and filter entries with missing crucial data
    df = pd.DataFrame(data)
    df = df[df['Number'].notna() & df['Name'].notna()]  # Ensure every entry has at least a number and a name
    return df

# Open the PDF and read its content
filename = 'data/biel_dhi_me_results_tt.pdf'
doc = fitz.open(filename)

all_text = []
for page in doc:
    text = page.get_text("text")
    all_text.extend(text.split('\n'))

# Parse the data directly from PDF text
df_timed_training_final = parse_timed_training_data_final(all_text)

# Extract runs and splits into a new structure
run_data = []
for _, row in df_timed_training_final.iterrows():
    for run in range(1, 4):
        run_splits = row.get(f'Run{run}_Splits', [])
        if run_splits:
            run_entry = {
                'Number': row['Number'],
                'Name': row['Name'],
                'Run': run,
                'Speed': row.get(f'Run{run}_Speed', None),
                'Splits': run_splits if isinstance(run_splits, list) else []
            }
            run_data.append(run_entry)

# Convert to DataFrame
df_runs = pd.DataFrame(run_data)

# Add original split times to the DataFrame
for i in range(5):  # Assuming max_splits is 5
    df_runs[f'Orig_Split_{i+1}_Time'] = df_runs['Splits'].apply(lambda x: x[i] if i < len(x) else None)

# Convert split times to seconds
def convert_to_seconds(split_time):
    if pd.isna(split_time):
        return None
    if ':' in split_time:
        minutes, seconds = map(float, split_time.split(':'))
        return minutes * 60 + seconds
    return float(split_time)

# Calculate and add cleaned split times to the DataFrame
for i in range(5):
    df_runs[f'Clean_Split_{i+1}_Time'] = df_runs[f'Orig_Split_{i+1}_Time'].apply(convert_to_seconds)

# Define function to calculate sector times
def calculate_sector_times(splits):
    sectors = []
    previous_split_time = 0
    for split in splits:
        # Convert split time to seconds
        split_time = convert_to_seconds(split)
        sector_time = split_time - previous_split_time if split_time is not None else None
        sectors.append(sector_time)
        previous_split_time = split_time if split_time is not None else previous_split_time
    return sectors

# Calculate and add sector times to the DataFrame
df_runs['Sector_Times'] = df_runs['Splits'].apply(lambda x: calculate_sector_times(x) if isinstance(x, list) else [])

# Ensure each run has 5 sectors
def pad_sector_times(sector_times):
    if len(sector_times) < 5:
        sector_times.extend([None] * (5 - len(sector_times)))
    return sector_times

df_runs['Sector_Times'] = df_runs['Sector_Times'].apply(pad_sector_times)

# Flatten the sector times into separate columns
for i in range(5):
    df_runs[f'Sector_{i+1}_Time'] = df_runs['Sector_Times'].apply(lambda x: x[i])

# Rank each split independently as integers, handling NaNs
for i in range(5):
    rank_col = f'Split_{i+1}_Rank'
    df_runs[rank_col] = df_runs[f'Clean_Split_{i+1}_Time'].rank(method='min', na_option='bottom')
    df_runs[rank_col] = df_runs[rank_col].fillna(df_runs[rank_col].max() + 1).astype(int)

# Rank each sector independently as integers, handling NaNs
for i in range(5):
    rank_col = f'Sector_{i+1}_Rank'
    df_runs[rank_col] = df_runs[f'Sector_{i+1}_Time'].rank(method='min', na_option='bottom')
    df_runs[rank_col] = df_runs[rank_col].fillna(df_runs[rank_col].max() + 1).astype(int)

# Calculate the total time for each run by summing the sector times
df_runs['Total_Time'] = df_runs[[f'Sector_{i+1}_Time' for i in range(5)]].sum(axis=1)

# Rank the overall performance of each run based on the total time, handling NaNs
df_runs['Overall_Rank'] = df_runs['Total_Time'].rank(method='min', na_option='bottom')
df_runs['Overall_Rank'] = df_runs['Overall_Rank'].fillna(df_runs['Overall_Rank'].max() + 1).astype(int)

# Display the final DataFrame with ranks and total times
print(df_runs.head())
print(df_runs.tail())


  Number          Name  Run   Speed  \
0      8  BROSNAN Troy    1  53.824   
1      8  BROSNAN Troy    2  57.353   
2      8  BROSNAN Troy    3  58.137   
3     27  THIRION Rémi    1  55.847   
4     27  THIRION Rémi    2  53.578   

                                              Splits Orig_Split_1_Time  \
0  [0:45.070, 1:43.159, 2:12.260, 4:41.049, 5:21....          0:45.070   
1  [0:44.328, 1:39.179, 4:42.540, 5:30.841, 5:58....          0:44.328   
2  [0:43.300, 1:36.057, 2:03.263, 2:50.291, 3:17....          0:43.300   
3  [0:45.759, 2:57.869, 3:25.891, 4:22.702, 10:00...          0:45.759   
4  [0:42.594, 1:35.880, 2:02.818, 2:54.489, 3:21....          0:42.594   

  Orig_Split_2_Time Orig_Split_3_Time Orig_Split_4_Time Orig_Split_5_Time  \
0          1:43.159          2:12.260          4:41.049          5:21.632   
1          1:39.179          4:42.540          5:30.841          5:58.372   
2          1:36.057          2:03.263          2:50.291          3:17.247   
3          2

In [7]:
import pandas as pd
import re
import fitz  # PyMuPDF

filename = 'data/biel_dhi_me_results_tt.pdf'

def parse_timed_training_data_final(lines):
    data = []
    current_rider = {}
    run_number = 0
    collect_splits = False
    start_collecting = False  # Flag to start collecting data

    for line in lines:
        line = line.strip()
        if 'Nr Name / UCI MTB Team' in line:  # This marks the start of rider data
            start_collecting = True
            continue
        if not start_collecting:
            continue

        if line.endswith('.') or (re.match(r'^\d+ [A-Z]', line) and not current_rider):
            # Resetting for a new rider entry
            if current_rider:
                data.append(current_rider.copy())
                current_rider = {}
                run_number = 0
                collect_splits = False
            current_rider['Rank'] = line[:-1] if line.endswith('.') else None
            continue

        if re.match(r'^\d+ [A-Z]', line):  # Captures the number and name
            parts = line.split(maxsplit=1)
            current_rider['Number'] = parts[0]
            current_rider['Name'] = parts[1]
        elif line.isupper():  # Handles team and nationality
            if any(ext in line for ext in ['TEAM', 'FACTORY', 'RACING', 'GRAVITY']):
                current_rider['Team'] = line
            else:
                current_rider['NAT'] = line
        elif re.match(r'^\d+\.\d+$', line):  # Handles speed for a run
            run_number += 1
            current_rider[f'Run{run_number}_Speed'] = line
            collect_splits = True
            current_rider[f'Run{run_number}_Splits'] = []  # Prepare to collect splits
        elif collect_splits and (re.match(r'\d+:\d+\.\d+$', line) or re.match(r'\d+\.\d+$', line)):  # Captures splits
            current_rider[f'Run{run_number}_Splits'].append(line)
        elif collect_splits and re.match(r'\+\d+\.\d+$', line):  # Handles the delta time
            current_rider[f'Run{run_number}_Delta'] = line
            collect_splits = False  # Stop collecting splits for this run

    # Append the last rider if not already appended
    if current_rider:
        data.append(current_rider)

    # Convert to DataFrame and filter entries with missing crucial data
    df = pd.DataFrame(data)
    df = df[df['Number'].notna() & df['Name'].notna()]  # Ensure every entry has at least a number and a name
    return df

# Open the PDF and read its content

doc = fitz.open(filename)

all_text = []
for page in doc:
    text = page.get_text("text")
    all_text.extend(text.split('\n'))

# Parse the data directly from PDF text
df_timed_training_final = parse_timed_training_data_final(all_text)

# Extract runs and splits into a new structure
run_data = []
for _, row in df_timed_training_final.iterrows():
    for run in range(1, 4):
        run_splits = row.get(f'Run{run}_Splits', [])
        if run_splits:
            run_entry = {
                'Number': row['Number'],
                'Name': row['Name'],
                'Run': run,
                'Speed': row.get(f'Run{run}_Speed', None),
                'Splits': run_splits if isinstance(run_splits, list) else []
            }
            run_data.append(run_entry)

# Convert to DataFrame
df_runs = pd.DataFrame(run_data)

# Add original split times to the DataFrame
for i in range(5):  # Assuming max_splits is 5
    df_runs[f'Orig_Split_{i+1}_Time'] = df_runs['Splits'].apply(lambda x: x[i] if i < len(x) else None)

# Convert split times to seconds
def convert_to_seconds(split_time):
    if pd.isna(split_time):
        return None
    if ':' in split_time:
        minutes, seconds = map(float, split_time.split(':'))
        return minutes * 60 + seconds
    return float(split_time)

# Calculate and add cleaned split times to the DataFrame
for i in range(5):
    df_runs[f'Clean_Split_{i+1}_Time'] = df_runs[f'Orig_Split_{i+1}_Time'].apply(convert_to_seconds)

# Define function to calculate sector times
def calculate_sector_times(splits):
    sectors = []
    previous_split_time = 0
    for split in splits:
        # Convert split time to seconds
        split_time = convert_to_seconds(split)
        sector_time = split_time - previous_split_time if split_time is not None else None
        sectors.append(sector_time)
        previous_split_time = split_time if split_time is not None else previous_split_time
    return sectors

# Calculate and add sector times to the DataFrame
df_runs['Sector_Times'] = df_runs['Splits'].apply(lambda x: calculate_sector_times(x) if isinstance(x, list) else [])

# Ensure each run has 5 sectors
def pad_sector_times(sector_times):
    if len(sector_times) < 5:
        sector_times.extend([None] * (5 - len(sector_times)))
    return sector_times

df_runs['Sector_Times'] = df_runs['Sector_Times'].apply(pad_sector_times)

# Flatten the sector times into separate columns
for i in range(5):
    df_runs[f'Sector_{i+1}_Time'] = df_runs['Sector_Times'].apply(lambda x: x[i])

# Rank each split independently as integers, handling NaNs
for i in range(5):
    rank_col = f'Split_{i+1}_Rank'
    df_runs[rank_col] = df_runs[f'Clean_Split_{i+1}_Time'].rank(method='min', na_option='bottom')
    df_runs[rank_col] = df_runs[rank_col].fillna(df_runs[rank_col].max() + 1).astype(int)

# Rank each sector independently as integers, handling NaNs
for i in range(5):
    rank_col = f'Sector_{i+1}_Rank'
    df_runs[rank_col] = df_runs[f'Sector_{i+1}_Time'].rank(method='min', na_option='bottom')
    df_runs[rank_col] = df_runs[rank_col].fillna(df_runs[rank_col].max() + 1).astype(int)

# Convert speed to float for ranking and handle NaNs
df_runs['Speed'] = df_runs['Speed'].astype(float).fillna(0)  # Replace NaNs with a very small number (0)

# Rank speeds in descending order (higher speeds are better)
df_runs['Speed_Rank'] = df_runs['Speed'].rank(method='min', ascending=False, na_option='bottom')
df_runs['Speed_Rank'] = df_runs['Speed_Rank'].fillna(df_runs['Speed_Rank'].max() + 1).astype(int)

# Calculate cumulative times from each split to the finish
for i in range(4):  # Only up to Split 4
    df_runs[f'Cumulative_from_Split_{i+1}_Time'] = df_runs['Clean_Split_5_Time'] - df_runs[f'Clean_Split_{i+1}_Time']

# Rank cumulative times
for i in range(4):  # Only up to Split 4
    rank_col = f'Cumulative_from_Split_{i+1}_Rank'
    df_runs[rank_col] = df_runs[f'Cumulative_from_Split_{i+1}_Time'].rank(method='min', na_option='bottom')
    df_runs[rank_col] = df_runs[rank_col].fillna(df_runs[rank_col].max() + 1).astype(int)

# Save to CSV
# split filename and extension then replace extension with .csv
output_filename = filename.rsplit('.', 1)[0] + '.csv'
df_runs.to_csv(output_filename, index=False)

print(f'Data saved to {output_filename}')


Data saved to data/biel_dhi_me_results_tt.csv


In [8]:
df_runs.head(5)

,Number,Name,Run,Speed,Splits,Orig_Split_1_Time,Orig_Split_2_Time,Orig_Split_3_Time,Orig_Split_4_Time,Orig_Split_5_Time,...,Sector_5_Rank,Speed_Rank,Cumulative_from_Split_1_Time,Cumulative_from_Split_2_Time,Cumulative_from_Split_3_Time,Cumulative_from_Split_4_Time,Cumulative_from_Split_1_Rank,Cumulative_from_Split_2_Rank,Cumulative_from_Split_3_Rank,Cumulative_from_Split_4_Rank
0,8,BROSNAN Troy,1,53.824,"[0:45.070, 1:43.159, 2:12.260, 4:41.049, 5:21....",0:45.070,1:43.159,2:12.260,4:41.049,5:21.632,...,248,227,276.562,218.473,189.372,40.583,146,199,254,248
1,8,BROSNAN Troy,2,57.353,"[0:44.328, 1:39.179, 4:42.540, 5:30.841, 5:58....",0:44.328,1:39.179,4:42.540,5:30.841,5:58.372,...,18,15,314.044,259.193,75.832,27.531,185,230,4,18
2,8,BROSNAN Troy,3,58.137,"[0:43.300, 1:36.057, 2:03.263, 2:50.291, 3:17....",0:43.300,1:36.057,2:03.263,2:50.291,3:17.247,...,8,5,153.947,101.190,73.984,26.956,1,1,1,8
3,27,THIRION Rémi,1,55.847,"[0:45.759, 2:57.869, 3:25.891, 4:22.702, 10:00...",0:45.759,2:57.869,3:25.891,4:22.702,10:00.182,...,322,74,554.423,422.313,394.291,337.480,288,298,313,322
4,27,THIRION Rémi,2,53.578,"[0:42.594, 1:35.880, 2:02.818, 2:54.489, 3:21....",0:42.594,1:35.880,2:02.818,2:54.489,3:21.470,...,9,241,158.876,105.590,78.652,26.981,5,10,18,9
